In [4]:
# Load the dataset from the data folder
setwd("../data")
iris.data <- read.csv("iris.data.txt", header = TRUE)

In [6]:
# Rename variables for simplicity
names(iris.data)[names(iris.data) == "sepalenght"] <- "sl"
names(iris.data)[names(iris.data) == "petalenght"] <- "pl"
names(iris.data)[names(iris.data) == "sepalwidth"] <- "sw"
names(iris.data)[names(iris.data) == "petalwidth"] <- "pw"

In [8]:
# Convert 'class' to numeric factor
iris.data$class = as.factor(iris.data$class)
iris.data$class = as.numeric(iris.data$class)

In [10]:
# Fit first linear model
m1 = lm(class ~ ., data = iris.data)
summary(m1)


Call:
lm(formula = class ~ ., data = iris.data)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.59046 -0.15230  0.01338  0.10332  0.55061 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.19208    0.20470   5.824 3.57e-08 ***
sl          -0.10974    0.05776  -1.900 0.059418 .  
sw          -0.04424    0.05996  -0.738 0.461832    
pl           0.22700    0.05699   3.983 0.000107 ***
pw           0.60989    0.09447   6.456 1.52e-09 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2191 on 145 degrees of freedom
Multiple R-squared:  0.9304,	Adjusted R-squared:  0.9285 
F-statistic: 484.8 on 4 and 145 DF,  p-value: < 2.2e-16


In [ ]:
shapiro.test(residuals(m1))
# R2 = 0.9285, p-value = 0.4589

In [12]:
# Add interaction terms to improve R2
m2 = lm(class ~ . + sl*sw + pl*pw, data = iris.data)
summary(m2)


Call:
lm(formula = class ~ . + sl * sw + pl * pw, data = iris.data)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.55036 -0.11915 -0.00777  0.11794  0.56971 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.54327    1.04585   1.476 0.142246    
sl          -0.10167    0.18033  -0.564 0.573786    
sw          -0.04040    0.34467  -0.117 0.906865    
pl           0.17910    0.05618   3.188 0.001758 ** 
pw           0.21188    0.13803   1.535 0.126989    
sl:sw       -0.01416    0.05855  -0.242 0.809199    
pl:pw        0.08617    0.02241   3.845 0.000181 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2099 on 143 degrees of freedom
Multiple R-squared:  0.937,	Adjusted R-squared:  0.9343 
F-statistic: 354.3 on 6 and 143 DF,  p-value: < 2.2e-16


In [ ]:
shapiro.test(residuals(m2))

In [ ]:
qqnorm(m2$residuals)
qqline(m2$residuals, col = 'pink') # Tails are quite distant
# R2 = 0.9343, p-value = 0.105

In [ ]:
# Analyze influential points in m2

# Plot residuals
plot(m2$res, ylab = "Residuals", main = "Plot of residuals")
sort(m2$res)
sort(m2$res)[c(1, 50)]  # Check the first and last residual

In [16]:
# 1. Leverage points (R2 = 0.9464, p-value = 7.6e-05)
r = m2$rank
p = 4
n = dim(iris.data)[1]
lev = hatvalues(m2)
watchout_points_lev = lev[which(lev > 2 * r / n)]
watchout_ids_lev = seq_along(lev)[which(lev > 2 * r / n)]

# Fit model without leverage points
id_to_keep = !(1:n %in% watchout_ids_lev)
m3 = lm(class ~ . + sl*pl + sl*sw + sl*pw + pl*sw + pl*pw + sw*pw, iris.data[id_to_keep, ])
summary(m3)
shapiro.test(residuals(m3))


Call:
lm(formula = class ~ . + sl * pl + sl * sw + sl * pw + pl * sw + 
    pl * pw + sw * pw, data = iris.data[id_to_keep, ])

Residuals:
     Min       1Q   Median       3Q      Max 
-0.59122 -0.09450  0.00592  0.06546  0.58213 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)   
(Intercept)  5.78579    2.09196   2.766  0.00651 **
sl          -1.03868    0.45991  -2.258  0.02558 * 
sw          -1.38772    0.62276  -2.228  0.02758 * 
pl          -0.04118    0.52246  -0.079  0.93730   
pw           1.81691    1.14235   1.591  0.11415   
sl:pl        0.04358    0.05143   0.847  0.39841   
sl:sw        0.29290    0.13093   2.237  0.02698 * 
sl:pw       -0.10722    0.14163  -0.757  0.45037   
sw:pl       -0.03591    0.12999  -0.276  0.78281   
pl:pw        0.14288    0.04510   3.168  0.00191 **
sw:pw       -0.37807    0.26821  -1.410  0.16106   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.1909 on 130 degrees of freedom
Mul


	Shapiro-Wilk normality test

data:  residuals(m3)
W = 0.95735, p-value = 0.0002322


In [18]:
# 2. Cook's distance
Cdist = cooks.distance(m2)
watchout_ids_Cdist = which(Cdist > 4 / (n - p - 1))
watchout_Cdist = Cdist[watchout_ids_Cdist]

# Fit model without Cook's distance leverage points
id_to_keep = !(1:n %in% watchout_ids_Cdist)
m4 = lm(class ~ . + sl*pl + sl*sw + sl*pw + pl*sw + pl*pw + sw*pw, iris.data[id_to_keep, ])
summary(m4)
shapiro.test(residuals(m4))


Call:
lm(formula = class ~ . + sl * pl + sl * sw + sl * pw + pl * sw + 
    pl * pw + sw * pw, data = iris.data[id_to_keep, ])

Residuals:
     Min       1Q   Median       3Q      Max 
-0.55262 -0.09506  0.00340  0.06794  0.46329 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  2.852972   1.406696   2.028 0.044621 *  
sl          -0.323208   0.315323  -1.025 0.307292    
sw          -0.659186   0.415044  -1.588 0.114701    
pl          -0.218443   0.412644  -0.529 0.597463    
pw           1.988054   0.934703   2.127 0.035344 *  
sl:pl       -0.002298   0.042642  -0.054 0.957114    
sl:sw        0.110829   0.086700   1.278 0.203456    
sl:pw       -0.082269   0.122280  -0.673 0.502292    
sw:pl        0.082899   0.099617   0.832 0.406857    
pl:pw        0.164859   0.041143   4.007 0.000104 ***
sw:pw       -0.479223   0.222030  -2.158 0.032765 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.1668 on 12


	Shapiro-Wilk normality test

data:  residuals(m4)
W = 0.95354, p-value = 0.0001245


In [20]:
# 3. Standardized residuals
res_std = m2$res / summary(m2)$sigma
watchout_ids_rstd = which(abs(res_std) > 2)

# Fit model without standardized residuals
id_to_keep = !(1:n %in% watchout_ids_rstd)
m5 = lm(class ~ . + sl*pl + sl*sw + sl*pw + pl*sw + pl*pw + sw*pw, iris.data[id_to_keep, ])
summary(m5)
shapiro.test(residuals(m5))


Call:
lm(formula = class ~ . + sl * pl + sl * sw + sl * pw + pl * sw + 
    pl * pw + sw * pw, data = iris.data[id_to_keep, ])

Residuals:
     Min       1Q   Median       3Q      Max 
-0.39294 -0.08345  0.00182  0.07012  0.39567 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  4.391816   1.019102   4.309 3.25e-05 ***
sl          -0.759441   0.245707  -3.091 0.002453 ** 
sw          -1.042821   0.307347  -3.393 0.000922 ***
pl           0.083382   0.342548   0.243 0.808076    
pw           1.490784   0.771482   1.932 0.055541 .  
sl:pl       -0.002264   0.035459  -0.064 0.949190    
sl:sw        0.219537   0.067909   3.233 0.001562 ** 
sl:pw       -0.021470   0.101955  -0.211 0.833555    
sw:pl        0.014228   0.086222   0.165 0.869197    
pl:pw        0.137373   0.033514   4.099 7.35e-05 ***
sw:pw       -0.432271   0.187680  -2.303 0.022891 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.1458 on 12


	Shapiro-Wilk normality test

data:  residuals(m5)
W = 0.98322, p-value = 0.08855


In [ ]:
# 4. Studentized residuals
stud = rstandard(m2)
watchout_ids_stud = which(abs(stud) > 2)

# Fit model without studentized residuals
id_to_keep = !(1:n %in% watchout_ids_stud)
m6 = lm(class ~ . + sl*pl + sl*sw + sl*pw + pl*sw + pl*pw + sw*pw, iris.data[id_to_keep, ])
summary(m6)
shapiro.test(residuals(m6))

In [ ]:
# Model comparison
# Chosen model: m5 with standardized residuals (R2 = 0.9678, p-value = 0.08855)

qqnorm(m5$residuals)
qqline(m5$residuals, col = 'blue') # Tails are less distant

In [ ]:
# Plot standardized residuals (needs correction)
res_std <- rstandard(m5) 
plot(m5$fitted.values, res_std, ylab = "Standardized Residuals", main = "Standardized Residuals")
abline(h = c(-2, 2), lty = 2, col = 'orange')
points(m5$fitted.values[watchout_ids_rstd], 
       res_std[watchout_ids_rstd], col = 'red', pch = 16)
points(m5$fitted.values[watchout_ids_lev], 
       res_std[watchout_ids_lev], col = 'orange', pch = 16)
legend('topright', col = c('red', 'orange'), 
       legend = c('Standardized Residuals', 'Leverages'), pch = rep(16, 2), bty = 'n')

In [ ]:
# Stepwise covariate selection
m5 = lm(class ~ . + sl*pl + sl*sw + sl*pw + pl*sw + pl*pw + sw*pw, iris.data[id_to_keep, ])
summary(m5)
shapiro.test(residuals(m5))

In [ ]:
step(m5, direction = "both", trace = TRUE)
m7 = lm(formula = class ~ sl + sw + pl + pw + sl:sw + pl:pw + sw:pw, 
        data = iris.data[id_to_keep, ])
summary(m7)
shapiro.test(residuals(m7))

In [ ]:
# After stepwise selection: R2 = 0.9685, p-value = 0.1171

# Manually remove 'pw' (as it has the largest estimate)
m8 = lm(formula = class ~ sl + sw + pl + sl:sw + pl:pw + sw:pw, 
        data = iris.data[id_to_keep, ])
summary(m8)
shapiro.test(residuals(m8))
# Final model: R2 = 0.959, p-value = 0.4222 (indicating normality of residuals).

In [ ]:
# Q-Q plot for residuals
qqnorm(m8$residuals)
qqline(m8$residuals, col = 'violet')

# The model has reached a very good level (good fit).